In [1]:
import numpy as np
import pandas as pd
import boundary_condition_creator as bcc
import ventilation_wall as vw
import heat_transfer_coefficient as htc

# 簡易計算法における境界条件の検討
地域区分別の外界条件等に基づいた通気層の対流熱伝達率、通気層の放射熱伝達率の設定方法について検討する。

## 設定手順
#### 1. 地域区分別の外気条件データから、各時刻の傾斜面日射量を計算する。
    - 必要な外気条件データは、1時間毎の外気温度、法線面直達日射量、水平面天空日射量、太陽高度、太陽方位角。
    - 傾斜角は外壁、傾斜屋根、陸屋根を想定し、90度、30度、0度とする。
    - 傾斜面の方位に依存しない日射量とするため、円柱面を想定した日射量を計算する（傾斜面方位角＝太陽方位角として傾斜面直達日射量を計算する）
    - 地域区分別の傾斜面日射量を計算し、CSVファイルとして保存しておく。
#### 2. 外気条件データを元に、地域区分別の冬期、夏期の日照時間帯の平均外気温度、平均傾斜面日射量を計算する。
    - 冬期は1月、夏期は8月の外気条件データとする。
    - 日照時間帯は、傾斜面日射量が0より大きい時間帯とする。
#### 3. 平均外気温度、平均傾斜面日射量を境界条件（パラメータ）として、通気層内の面1、面2の表面温度を計算する。
    - 外気温度、傾斜面日射量以外のパラメータは、パラメトリックスタディに用いた値の平均値とする。
#### 4. 通気層内の面1、面2の表面温度を元に、通気層の対流熱伝達率、通気層の放射熱伝達率を計算する。
    4.1 対流熱伝達率の計算：
        - 通気層の傾斜角、通気層の厚さは通気層の仕様なので、入力条件として与えられることを想定する。
        - 通気層内の平均風速は、既往研究等を参考に設定する。
    4.2 放射熱伝達率の計算：
        - 通気層に面する面の放射率は通気層の仕様なので、入力条件として与えられることを想定する。

## 1. 各時刻の傾斜面日射量を計算する
### 外気条件データの例（1地域）

In [2]:
# CSVファイルを読み込む
df = pd.read_csv('climateData/rev_climateData_1.csv', index_col=0, encoding="shift-jis")
df.query("月 == 1 & 法線面直達日射量_W_m2 > 0")

,日,時,外気温_degree,外気絶対湿度_kg_kgDA,法線面直達日射量_W_m2,水平面天空日射量_W_m2,水平面夜間放射量_W_m2,太陽高度角_度,太陽方位角_度,傾斜面日射量_0度_W_m2,傾斜面日射量_30度_W_m2,傾斜面日射量_90度_W_m2
月,,,,,,,,,,,,
1,1,7,-17.5,0.0009,2.322222,5.805556,61.538889,-1,-58,5.765027,6.619729,5.801149
1,1,8,-16.7,0.0010,277.505556,44.122222,61.538889,8,-47,82.743531,213.124627,305.140355
1,1,9,-15.6,0.0011,613.066667,61.538889,82.438889,15,-35,220.212218,493.870447,644.967593
1,1,10,-11.6,0.0015,818.583333,52.250000,90.566667,20,-21,332.221989,680.272058,828.563917
1,1,11,-11.6,0.0015,846.450000,58.055556,90.566667,23,-7,388.789919,735.380395,847.068103
...,...,...,...,...,...,...,...,...,...,...,...,...
1,31,12,-10.1,0.0017,892.894444,61.538889,90.566667,28,6,480.727439,821.074525,867.221189
1,31,13,-7.7,0.0018,876.638889,58.055556,92.888889,26,22,442.348751,786.859497,861.180467
1,31,14,-6.0,0.0017,796.522222,58.055556,97.533333,20,36,330.482200,668.765615,810.562052


## 2. 地域区分別の冬期、夏期の平均外気温度、平均傾斜面日射量を計算する
- 冬期（1月）、夏期（8月）の日照時間帯の平均外気温度、平均傾斜面日射量を計算する

In [3]:
# 外壁（傾斜角90度）
df_90 = bcc.get_all_region_climate_conditions(angle_list=[90])

In [4]:
# 傾斜屋根（傾斜角30度）
df_30 = bcc.get_all_region_climate_conditions(angle_list=[30])

In [5]:
# 陸屋根（傾斜角0度）
df_0 = bcc.get_all_region_climate_conditions(angle_list=[0])

## 3. 通気層内の面1、面2の表面温度を計算する
- 外気温度、傾斜面日射量以外のパラメータは、パラメトリックスタディに用いた値の平均値とする。

In [6]:
# 外壁（傾斜角90度）：夏期
df_90 = bcc.add_ventilation_wall_temperatures_and_heat_flow(df_90)
df_90.query('season == "summer"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
1,1,summer,90,22.43,281.34,26.97,25.38,23.27,18.22,21.43,-3.22
3,2,summer,90,23.06,251.45,27.11,25.67,23.79,16.24,18.88,-2.65
5,3,summer,90,24.94,270.38,29.19,27.16,25.69,19.33,19.01,0.32
7,4,summer,90,26.53,277.81,30.81,28.39,27.26,21.18,18.42,2.77
9,5,summer,90,26.19,240.21,29.93,27.92,26.83,17.81,15.99,1.82
11,6,summer,90,28.71,295.10,33.16,30.15,29.43,24.28,18.03,6.26
13,7,summer,90,28.51,317.35,33.30,30.13,29.30,26.00,19.77,6.22
15,8,summer,90,28.90,300.85,33.43,30.33,29.64,24.91,18.29,6.62


In [7]:
# 外壁（傾斜角90度）：冬期
df_90.query('season == "winter"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
0,1,winter,90,-7.89,258.19,-3.09,0.62,-6.32,6.13,44.63,-38.50
2,2,winter,90,-4.57,271.10,0.37,3.28,-3.10,8.04,41.25,-33.22
4,3,winter,90,-1.18,280.55,3.85,5.98,0.17,9.67,37.53,-27.86
6,4,winter,90,0.44,346.30,6.52,7.74,1.81,14.11,38.46,-24.35
8,5,winter,90,4.30,457.50,12.08,11.38,5.75,22.91,40.03,-17.12
10,6,winter,90,5.96,335.44,11.84,12.09,7.02,13.78,29.51,-15.73
12,7,winter,90,9.58,394.28,16.09,14.69,10.82,22.90,33.44,-10.54
14,8,winter,90,17.98,205.81,21.25,19.76,18.53,14.04,14.51,-0.47


In [8]:
# 傾斜屋根（傾斜角30度）：夏期
df_30 = bcc.add_ventilation_wall_temperatures_and_heat_flow(df_30)
df_30.query('season == "summer"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
1,1,summer,30,22.43,358.46,28.08,25.76,23.53,25.39,27.85,-2.46
3,2,summer,30,23.06,338.98,28.39,26.12,24.08,24.21,25.95,-1.75
5,3,summer,30,24.94,366.17,30.59,27.70,25.99,27.96,26.58,1.39
7,4,summer,30,26.53,388.79,32.44,29.04,27.61,31.09,27.04,4.06
9,5,summer,30,26.19,336.83,31.35,28.47,27.13,26.39,23.46,2.93
11,6,summer,30,28.71,407.78,34.81,30.84,29.79,34.35,26.73,7.62
13,7,summer,30,28.51,438.97,35.07,30.87,29.69,36.91,29.22,7.69
15,8,summer,30,28.90,429.43,35.31,31.12,30.05,36.37,28.19,8.18


In [9]:
# 傾斜屋根（傾斜角30度）：冬期
df_30.query('season == "winter"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
0,1,winter,30,-7.89,234.60,-3.48,0.41,-6.35,4.63,43.56,-38.92
2,2,winter,30,-4.57,245.91,-0.04,3.04,-3.13,6.57,40.27,-33.70
4,3,winter,30,-1.18,273.60,3.72,5.84,0.18,9.64,37.78,-28.14
6,4,winter,30,0.44,330.80,6.22,7.50,1.83,13.85,38.68,-24.83
8,5,winter,30,4.30,424.17,11.56,11.19,5.68,20.55,38.05,-17.51
10,6,winter,30,5.96,328.99,11.68,11.91,7.06,14.24,30.31,-16.08
12,7,winter,30,9.58,394.66,16.05,14.60,10.87,23.83,34.56,-10.73
14,8,winter,30,17.98,246.37,21.84,19.96,18.66,17.81,17.88,-0.07


In [10]:
# 陸屋根（傾斜角0度）：夏期
df_0 = bcc.add_ventilation_wall_temperatures_and_heat_flow(df_0)
df_0.query('season == "summer"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
1,1,summer,0,22.43,293.63,27.10,25.38,23.34,20.07,23.29,-3.22
3,2,summer,0,23.06,292.51,27.68,25.84,23.95,20.46,22.77,-2.31
5,3,summer,0,24.94,313.99,29.80,27.37,25.85,23.73,22.99,0.74
7,4,summer,0,26.53,339.67,31.69,28.73,27.47,27.13,23.68,3.44
9,5,summer,0,26.19,297.54,30.75,28.23,27.03,23.27,20.83,2.44
11,6,summer,0,28.71,352.24,33.97,30.48,29.64,29.82,22.90,6.92
13,7,summer,0,28.51,375.54,34.12,30.47,29.51,31.69,24.80,6.89
15,8,summer,0,28.90,373.53,34.47,30.76,29.89,31.81,24.33,7.48


In [11]:
# 陸屋根（傾斜角0度）：冬期
df_0.query('season == "winter"').style.set_precision(2)

,region,season,angle,theta_e_ave,j_surf_ave,theta_1_surf,theta_2_surf,theta_as_ave,q_outer_flow,q_exhaust_flow,q_inner_flow
0,1,winter,0,-7.89,140.72,-4.94,-0.23,-6.55,-2.32,37.87,-40.19
2,2,winter,0,-4.57,148.00,-1.57,2.37,-3.33,-0.54,34.49,-35.03
4,3,winter,0,-1.18,179.92,2.23,5.17,-0.00,3.14,32.60,-29.47
6,4,winter,0,0.44,207.20,4.24,6.59,1.61,5.72,32.37,-26.65
8,5,winter,0,4.30,252.16,8.74,9.89,5.40,10.08,30.16,-20.08
10,6,winter,0,5.96,215.28,9.77,10.90,6.92,8.19,26.26,-18.07
12,7,winter,0,9.58,259.44,14.04,13.97,9.92,12.23,21.46,-12.01
14,8,winter,0,17.98,201.32,21.15,19.70,18.55,14.22,14.82,-0.60


## 4. 通気層の対流熱伝達率、通気層の放射熱伝達率を計算する
### 4.1 通気層内の対流熱伝達率の計算
- 通気層の傾斜角、通気層の厚さは通気層の仕様なので、入力条件として与えられることを想定する。
- 通気層内の平均風速は、既往研究等を参考に設定する。

### 仮の通気層の通気層の平均風速、通気層の長さ、通気層の厚さを設定

In [12]:
# 通気層の平均風速, m/s
v_a_array = [0.0, 0.5, 1.0]

# 通気層の長さ, m
l_h = 7.5

# 通気層の厚さ, m
l_d = 0.2

### 対流熱伝達率を計算

In [13]:
# 外壁（傾斜角90度）
for v_a in v_a_array:
    df_90['h_cv_' + str(v_a)] = [htc.convective_heat_transfer_coefficient_detailed(
        v_a=v_a, theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, angle=row.angle,
        l_h=l_h, l_d=l_d
    ) for row in df_90.itertuples()]

In [14]:
# 傾斜屋根（傾斜角30度）
for v_a in v_a_array:
    df_30['h_cv_' + str(v_a)] = [htc.convective_heat_transfer_coefficient_detailed(
        v_a=v_a, theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, angle=row.angle,
        l_h=l_h, l_d=l_d
    ) for row in df_30.itertuples()]

In [15]:
# 陸屋根（傾斜角0度）
for v_a in v_a_array:
    df_0['h_cv_' + str(v_a)] = [htc.convective_heat_transfer_coefficient_detailed(
        v_a=v_a, theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, angle=row.angle,
        l_h=l_h, l_d=l_d
    ) for row in df_0.itertuples()]

### 対流熱伝達率の計算結果

#### 外壁（傾斜角90度）：夏期

In [16]:
df_table = df_90.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
1,1,summer,90,1.7,3.7,5.7
3,2,summer,90,1.6,3.6,5.6
5,3,summer,90,1.8,3.8,5.8
7,4,summer,90,1.9,3.9,5.9
9,5,summer,90,1.8,3.8,5.8
11,6,summer,90,2.1,4.1,6.1
13,7,summer,90,2.1,4.1,6.1
15,8,summer,90,2.1,4.1,6.1


#### 外壁（傾斜角90度）：冬期

In [17]:
df_table = df_90.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
0,1,winter,90,2.4,4.4,6.4
2,2,winter,90,2.2,4.2,6.2
4,3,winter,90,2.0,4.0,6.0
6,4,winter,90,1.6,3.6,5.6
8,5,winter,90,1.3,3.3,5.3
10,6,winter,90,1.0,3.0,5.0
12,7,winter,90,1.7,3.7,5.7
14,8,winter,90,1.7,3.7,5.7


#### 傾斜屋根（傾斜角30度）：夏期

In [18]:
df_table = df_30.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
1,1,summer,30,2.1,4.1,6.1
3,2,summer,30,2.1,4.1,6.1
5,3,summer,30,2.2,4.2,6.2
7,4,summer,30,2.3,4.3,6.3
9,5,summer,30,2.2,4.2,6.2
11,6,summer,30,2.4,4.4,6.4
13,7,summer,30,2.4,4.4,6.4
15,8,summer,30,2.4,4.4,6.4


#### 傾斜屋根（傾斜角30度）：冬期

In [19]:
df_table = df_30.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
0,1,winter,30,2.5,4.5,6.5
2,2,winter,30,2.3,4.3,6.3
4,3,winter,30,2.1,4.1,6.1
6,4,winter,30,1.8,3.8,5.8
8,5,winter,30,1.3,3.3,5.3
10,6,winter,30,1.2,3.2,5.2
12,7,winter,30,1.9,3.9,5.9
14,8,winter,30,2.0,4.0,6.0


#### 陸屋根（傾斜角0度）：夏期

In [20]:
df_table = df_0.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
1,1,summer,0,2.0,4.0,6.0
3,2,summer,0,2.0,4.0,6.0
5,3,summer,0,2.2,4.2,6.2
7,4,summer,0,2.3,4.3,6.3
9,5,summer,0,2.2,4.2,6.2
11,6,summer,0,2.4,4.4,6.4
13,7,summer,0,2.4,4.4,6.4
15,8,summer,0,2.4,4.4,6.4


#### 陸屋根（傾斜角0度）：冬期

In [21]:
df_table = df_0.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_cv_0.0', 'h_cv_0.5', 'h_cv_1.0']]
df_table.style.set_precision(1)

,region,season,angle,h_cv_0.0,h_cv_0.5,h_cv_1.0
0,1,winter,0,2.8,4.8,6.8
2,2,winter,0,2.6,4.6,6.6
4,3,winter,0,2.4,4.4,6.4
6,4,winter,0,2.2,4.2,6.2
8,5,winter,0,1.8,3.8,5.8
10,6,winter,0,1.8,3.8,5.8
12,7,winter,0,0.9,2.9,4.9
14,8,winter,0,1.9,3.9,5.9


### 4.2 通気層内の放射熱伝達率の計算
- 通気層に面する面の放射率は通気層の仕様なので、入力条件として与えられることを想定する。

### 仮の放射率を設定

In [22]:
# 通気層に面する面1,面2の放射率[-]
emissivity_1 = 0.9
emissivity_2_array = [0.1, 0.9]

### 放射熱伝達率を計算

In [23]:
# 外壁（傾斜角90度）
for emissivity_2 in emissivity_2_array:
    
    # 有効放射率の計算
    effective_emissivity = htc.effective_emissivity_parallel(emissivity_1, emissivity_2) 

    # 対流熱伝達率の計算
    df_90['h_rv' + str(emissivity_2)] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_90.itertuples()]

In [24]:
# 傾斜屋根（傾斜角30度）
for emissivity_2 in emissivity_2_array:
    
    # 有効放射率の計算
    effective_emissivity = htc.effective_emissivity_parallel(emissivity_1, emissivity_2) 

    # 対流熱伝達率の計算
    df_30['h_rv' + str(emissivity_2)] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_30.itertuples()]

In [25]:
# 陸屋根（傾斜角0度）
for emissivity_2 in emissivity_2_array:
    
    # 有効放射率の計算
    effective_emissivity = htc.effective_emissivity_parallel(emissivity_1, emissivity_2) 

    # 対流熱伝達率の計算
    df_0['h_rv' + str(emissivity_2)] = [htc.radiative_heat_transfer_coefficient_detailed(
                    theta_1=row.theta_1_surf, theta_2=row.theta_2_surf, effective_emissivity=effective_emissivity
                ) for row in df_0.itertuples()]

### 放射熱伝達率の計算結果

#### 外壁（傾斜角90度）：夏期

In [26]:
df_table = df_90.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
1,1,summer,90,0.6,5.0
3,2,summer,90,0.6,5.0
5,3,summer,90,0.6,5.1
7,4,summer,90,0.6,5.1
9,5,summer,90,0.6,5.1
11,6,summer,90,0.6,5.3
13,7,summer,90,0.6,5.3
15,8,summer,90,0.6,5.3


#### 外壁（傾斜角90度）：冬期

In [27]:
df_table = df_90.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
0,1,winter,90,0.5,3.7
2,2,winter,90,0.5,3.9
4,3,winter,90,0.5,4.0
6,4,winter,90,0.5,4.1
8,5,winter,90,0.5,4.3
10,6,winter,90,0.5,4.3
12,7,winter,90,0.5,4.5
14,8,winter,90,0.6,4.7


#### 傾斜屋根（傾斜角30度）：夏期

In [28]:
df_table = df_30.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
1,1,summer,30,0.6,5.0
3,2,summer,30,0.6,5.0
5,3,summer,30,0.6,5.1
7,4,summer,30,0.6,5.2
9,5,summer,30,0.6,5.2
11,6,summer,30,0.6,5.3
13,7,summer,30,0.6,5.3
15,8,summer,30,0.6,5.3


#### 傾斜屋根（傾斜角30度）：冬期

In [29]:
df_table = df_30.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
0,1,winter,30,0.4,3.7
2,2,winter,30,0.5,3.8
4,3,winter,30,0.5,4.0
6,4,winter,30,0.5,4.1
8,5,winter,30,0.5,4.3
10,6,winter,30,0.5,4.3
12,7,winter,30,0.5,4.5
14,8,winter,30,0.6,4.7


#### 陸屋根（傾斜角0度）：夏期

In [30]:
df_table = df_0.query('season == "summer"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
1,1,summer,0,0.6,5.0
3,2,summer,0,0.6,5.0
5,3,summer,0,0.6,5.1
7,4,summer,0,0.6,5.2
9,5,summer,0,0.6,5.1
11,6,summer,0,0.6,5.3
13,7,summer,0,0.6,5.3
15,8,summer,0,0.6,5.3


#### 陸屋根（傾斜角0度）：冬期

In [31]:
df_table = df_0.query('season == "winter"')[
    ['region', 'season', 'angle', 'h_rv0.1', 'h_rv0.9']]
df_table.style.set_precision(1)

,region,season,angle,h_rv0.1,h_rv0.9
0,1,winter,0,0.4,3.7
2,2,winter,0,0.5,3.8
4,3,winter,0,0.5,3.9
6,4,winter,0,0.5,4.0
8,5,winter,0,0.5,4.2
10,6,winter,0,0.5,4.2
12,7,winter,0,0.5,4.4
14,8,winter,0,0.6,4.7
